In [12]:
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime

In [45]:
# ETAPE 1 connexion à l'API + récupération offres
"""
comme je n'arrivais pas à bien séparer les offres les unes des autres, j'ai choisi 1 résultat par page de manière temporaire 
à  voir ensuite comment améliorer performance
"""

def create_url(start_url="http://api.adzuna.com/v1/api/jobs/"):
    """
    concactène l'adresse url à partir des paramètres choisis
    comment améliorer pour que les paramètres soient correctement passés ?
    """
    url_list = []
    for page_number in range(1, 3):
        #start_url = "http://api.adzuna.com/v1/api/jobs/"
        country = "fr"
        part0_url = "/search/"
        number_page = str(page_number) 
        part1_url = "?app_id="
        api_id = "37fe08af"
        part2_url = "&app_key="
        api_key = "17c1369ea8fb68bc48f834aebc0bec53"
        part3_url = "&results_per_page="
        results_per_page = str(1)
        part4_url = "&what_and="
        what_and = "data%20engineer"
        part5_url = "&max_days_old="
        max_days_old = str(30)
        end_url = "&content-type=application/json"
        full_url = f"{start_url}{country}{part0_url}{number_page}{part1_url}{api_id}{part2_url}{api_key}{part3_url}{results_per_page}{part4_url}{what_and}{part5_url}{max_days_old}{end_url}"
        url_list.append(full_url)
    return url_list


def scrape_urls(url):
    """
    lance la requête sur l'api Adzuna pour chacunes des url crééés
    si le status_code est incorrect, il s'affiche et l'url liée s'affiche
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print("The request for {} was not successful : status {}".format(url, response.status_code))
        return None
    


In [77]:
class Company:
    def __init__(self):
        self.name = None
        self.sector = None
        self.location = None
        self.description = None
        
class JobOffer:
    """
    distinction floue entre 
        self.required_experience / self.education / self.qualifications 
    """
    def __init__(self):
        self.title = None
        self.id_source = None 
        self.company = None
        self.contract_type = None
        self.location = None
        self.salary = None
        self.remote_type = None
        self.starting_date = None
        self.required_experience = None
        self.education = None
        self.description = None
        self.qualifications = None
        self.publication_date = None
        self.url_direct_offer = None

class GetJobOffer():
    def __init__(self, full_info):
        #print("\n full_info ________\n", full_info,"________\n")
        #print("\n type ________\n", type(full_info),"________\n")
        self.list_info = full_info.split(",")
        #print("\n type(self.list_info)________\n", type(self.list_info),"________\n")
        #for i, x in enumerate(self.list_info):
        #    print("\t\ni :",i," x :",x,"\t\n")
        self.company = Company()
        self.job_offer = JobOffer()
        

    def get_job_details(self):
        #print("\n self.list_info________\n", self.list_info,"________\n")
        for element in self.list_info:
            print("\t\n element is ", element, "\t\n")
            #print("\t\n type(element) is ", type(element), "\t\n")
            if "title" in element:
            #    print("\n\n\n\t\tdo I get here \n\n\n")
                self.job_offer.title = element.split(":")[1]
            elif "\"id\"" in element:
            #   print("\n\nelement.split(\":\")[1]",element.split(":")[1])
                self.job_offer.id_source = element.split(":")[1]
            elif "location\":{\"display_name\"" in element:
                #problème même chose pour la company
                #print("does it go here??________-")
                self.location = element.split("\":\"")[1]
            elif "created" in element:
            #    print("\n\nelement.split(\":\")[0]",element.split(":")[0])
            #    print(type(element.split(":")[1]))
                print(element.split("\":\"")[1][:10])
                self.publication_date = element.split("\":\"")[1][:10]
                #self.publication_date = datetime.strptime(element.split("\":\"")[1][:10],"%Y-%m-%d")
        return self.job_offer 
        
    def get_company_details(self):
        for element in self.list_info:
            if "category\":{\"tag" in element:
                self.company.sector = element.split("\":\"")[1]
            #elif "__CLASS__\":\"Adzuna::API::Response::Company" in element:
            elif "\"display_name\"" in element:    
                self.company.name = element.split("\":\"")[1]
            elif "description" in element:
                 self.description = element.split(":")[1]
            elif "\"tag\"" in element:  
                self.company.sector = element.split("\":\"")[1]
        return self.company

In [78]:
url_list = create_url()
jobs_offers = [scrape_urls(url) for url in tqdm(url_list, desc="Accessing API")]

for offer in jobs_offers:
    print("\n__________OFFER ")
    print(offer)
    print("\n")
    jobs_ready_for_export = GetJobOffer(offer).get_job_details()
    companies_ready_for_export = GetJobOffer(offer).get_company_details()

Accessing API: 100%|█| 2/2 [00:02<00:00,  1.18s/it


__________OFFER 
{"__CLASS__":"Adzuna::API::Response::JobSearchResults","mean":71732.35,"count":3377,"results":[{"description":"Rejoindre Alptis, c\u2019est int\u00e9grer un groupe qui conjugue bienveillance et performance, o\u00f9 l'on joue collectif, dans lequel on peut grandir et vivre des projets enrichissants. L'aventure vous tente ? Vous \u00eates peut-\u00eatre l'un de nos futurs Talents  Nous recrutons pour notre si\u00e8ge \u00e0 Lyon un Data Engineer (H/F) en CDI au sein de la Direction Digitale et Transformation. Notre objectif : renforcer l'\u00e9quipe DATA du groupe Alptis pour mener \u00e0 bien l'ensemble des projets permettant \u00e0 Alptis de devenir DA\u2026","redirect_url":"https://www.adzuna.fr/land/ad/4569616973?se=8KoW9PzQ7hGqJ5EkS6QgFg&utm_medium=api&utm_source=37fe08af&v=F43979544EAF91A45F1DCA7DE2835DF912EE325E","company":{"__CLASS__":"Adzuna::API::Response::Company","display_name":"Alptis Pr\u00e9voyance et Sant\u00e9"},"created":"2024-02-15T08:26:41Z","latitud

In [79]:
print(vars(jobs_ready_for_export))
print(vars(companies_ready_for_export))

{'title': '"Data Engineer"', 'id_source': '"4553919472"', 'company': None, 'contract_type': None, 'location': None, 'salary': None, 'remote_type': None, 'starting_date': None, 'required_experience': None, 'education': None, 'description': None, 'qualifications': None, 'publication_date': None, 'url_direct_offer': None}
{'name': 'Paris', 'sector': 'unknown"', 'location': None, 'description': None}
